
<html>
	<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.1/css/bootstrap.min.css" integrity="sha384-VCmXjywReHh4PwowAiWNagnWcLhlEJLA5buUprzK8rxFgeH0kww/aWY76TfkUoSX" crossorigin="anonymous">
    <body>
		<div class="container-fluid">
			<h1> Multi-variable Coronavirus Projections for US States <img src='https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftoppng.com%2Fuploads%2Fthumbnail%2Fcoronavirus-covid-19-11582576817hpyucwma6o.png&f=1&nofb=1' width='50', height='50'> </h1>
			<p class="text-monospace"> This notebook aims to predict future coronvirus cases in US States. It will use 
			multiple variables as input. They include population density, testing, as well as mobility data.</p>
			<p class="text-monospace"> Sources:  </p>
			<ul class="list-group list-group-horizontal-lg">
			  <li class="list-group-item">Mobility data: <a href='https://www.apple.com/covid19/mobility'>Apple</a> </li>
			  <li class="list-group-item">COVID-19 tracking API: <a href='https://covidtracking.com/'> The Covid Tracking</a></li>
			  <li class="list-group-item">COVID-19 State Data: <a href='https://www.kaggle.com/nightranger77/covid19-state-data'> Nightranger77's Dataset</a></li>
                 <li class="list-group-item">COVID-19 Image: <a href='https://toppng.com/uploads/thumbnail/coronavirus-covid-19-11582576817hpyucwma6o.png'> Toppng.com
                     </a></li>
			</ul>
			<br>
			<b><p class="text-monospace">Feel free to provide me with feedback. I really want to hear about your ideas.</p></b>
            <p class="text-monospace">Longer outputs are being hidden.</p>
			<p class="text-monospace">Check out my visualization notebook <a href='https://www.kaggle.com/therealcyberlord/coronavirus-covid-19-visualization-prediction'>here</a>.</p>

    
<html>

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression, BayesianRidge, LinearRegression
import datetime
import matplotlib.pyplot as plt 
import warnings
import tensorflow as tf 
from scipy.stats import exponweib
%matplotlib inline

# Read the coronavirus dataset from the covid tracking API

In [ ]:
data = pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')
data.head()

Checking out the data for null values 

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

# Sort values by date since we are approaching it in a chronological order

In [ ]:
data.sort_values('date', inplace=True)

# Handling null values 

In [ ]:
# drop columns with 50% null values 
def drop_na_50(df):
    columns = df.columns
    for col in columns:
       if data[col].isnull().sum() > 0.5 * len(df):
         df.drop(col, 1, inplace=True)
    return df 
         
new_data = drop_na_50(data)

In [ ]:
new_data.isnull().sum()

In [ ]:
new_data.fillna(0, inplace=True)

# Let's look at Connecticut as an example. 

In [ ]:
new_data[new_data.state=='CT'].hist(bins=10, figsize=(20, 15))
plt.show()

# Drop the features we don't need 

In [ ]:
# drop variables with few/no variations
new_data.drop(['commercialScore', 'fips', 'hospitalizedIncrease', 'negativeRegularScore', 'negativeScore', 'positiveScore', 'score'], axis=1, inplace=True)

In [ ]:
new_data[new_data.state=='CT'].hist(bins=10, figsize=(20, 15))
plt.show()

In [ ]:
new_data.isnull().sum()

We need a way to map states to their abbreviations. This will help us later in the next dataset.

In [ ]:
# dictonary for mapping state to abbreviation 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}


# Again, let's use Connecticut as an example. It is an amazing state. 

In [ ]:
us_state_abbrev['Connecticut']

# Load state information, we want the population density from each state from this. 

In [ ]:
# this data frame is going to provide static variables
additional_data = pd.read_csv('../input/covid19-state-data/COVID19_state.csv')
additional_data.head()

In [ ]:
additional_data.isnull().sum()

In [ ]:
# additional_data.hist(bins=10, figsize=(20, 15))
# plt.show()

# Use the dictionary we created before to ensure that data are consistent 

In [ ]:
# make the data consistent with the other data frame 
for i in additional_data.State.unique():
    additional_data.replace(i, us_state_abbrev[i], inplace=True)

# Let's take a look at California's population density. 

In [ ]:
additional_data[additional_data.State=='CA']['Pop Density']

In [ ]:
# incoporate population density into prediction 
pop_dense = [] 
for i in new_data.state:
    pop_dense.append(additional_data[additional_data.State==i]['Pop Density'].sum())

In [ ]:
new_data['pop_dense'] = pop_dense

In [ ]:
new_data['pop_dense'].head()

# Scale the popluation density between 0 and 1. It is always good to normalize your data.

In [ ]:
# scales the population density based on all the states 
min_max_scaler = MinMaxScaler()
new_data['pop_dense'] = min_max_scaler.fit_transform(np.array(new_data['pop_dense']).reshape(-1, 1))

In [ ]:
new_data['pop_dense'].head()

Take a look at Apple's mobility data. 

In [ ]:
#this dataset is going to provide mobility information within the US

mobility_data = pd.read_csv('https://covid19-static.cdn-apple.com/covid19-mobility-data/2014HotfixDev17/v3/en-us/applemobilitytrends-2020-08-16.csv')
mobility_data.head()

In [ ]:
mobility_data.isnull().sum()

In [ ]:
# make the data consistent with the other data frame 
for i in mobility_data['sub-region'].unique():
    if i in us_state_abbrev.keys():
        mobility_data.replace(i, us_state_abbrev[i], inplace=True)

In [ ]:
mobility_data['transportation_type'].unique()

# We want to make sure the dates from the mobility dataset matches with others. 

In [ ]:
# get individual states and dates
unique_states = new_data.state.unique()
unique_states.sort()
unique_dates = new_data.date.unique()

# making sure that the dates match between mobility and testing/cases
mobility_latest_date = datetime.datetime.strptime(mobility_data.columns[-1], '%Y-%m-%d').strftime('%Y%m%d')
mobility_latest_index = np.where(unique_dates == int(mobility_latest_date))[0][0]

# start from a later date 3/1/2020
unique_dates = unique_dates[39:mobility_latest_index+1]

Simple function to get mobility of a state at a particular day.

In [ ]:
# gets the mobility information of a particular day
def get_mobility_by_state(transport_type, state, day):
    return mobility_data[mobility_data['sub-region']==state][mobility_data['transportation_type']==transport_type].sum()[day]

In [ ]:
get_mobility_by_state('walking', 'FL', '2020-03-01')

# We also want the date format to match with others as well 

In [ ]:
# change the date format to match the mobility data 
revised_unique_dates = [] 
for i in range(len(unique_dates)):
    revised_unique_dates.append(datetime.datetime.strptime(str(unique_dates[i]), '%Y%m%d').strftime('%Y-%m-%d'))
revised_unique_dates

Take a look at Florida's mobility at a random day.

In [ ]:
print(get_mobility_by_state('transit', 'FL', revised_unique_dates[9]))

# We want to convert our dates into numbers. Our machine learning algorithm cannot take text.

In [ ]:
def convert_date_to_int(d):
    return [i for i in range(len(revised_unique_dates))]

days_since_3_1 = convert_date_to_int(revised_unique_dates)

# We are predicting 10 days. Extrapolation way into the future may not yield the most accurate results. 

In [ ]:
days_ahead = 10
future_dates = [i for i in range(len(revised_unique_dates)+days_ahead)]

# This is a SVM implementation I wrote. Feel free to use this instead of bayesian ridge. 

In [ ]:
def svm_reg(X_train, X_test, y_train, y_test, future_forecast, state):
        
    svm_confirmed = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, degree=4, C=0.1)
    svm_confirmed.fit(X_train, y_train)
    test_svm_pred = svm_confirmed.predict(X_test)
    svm_pred = svm_confirmed.predict(future_forecast)
    
    plt.plot(y_test)
    plt.plot(test_svm_pred)
    plt.title('Testing Set Evaluation for {}'.format(state))
    plt.xlabel('Days since 3/1/2020')
    plt.ylabel('# of positive coronavirus cases')
    plt.legend(['Actual', 'Predicted'])
    plt.show()
    
    print('MAE:', mean_absolute_error(test_svm_pred, y_test))
    print('MSE:',mean_squared_error(test_svm_pred, y_test))

    # plot the graph to see compare predictions and actual coronavirus cases
    plt.plot(positive)
    plt.plot(svm_pred)
    plt.title('Coronavirus Cases in {}'.format(state))
    plt.legend(['Actual cases', 'Predicted cases using support vector regression'])
    plt.xlabel('Days since 3/1/2020')
    plt.ylabel('# of positive coronavirus cases')
    plt.show()
    print('Completed:', state)


# Bayesian ridge implementation of the prediction model.

In [ ]:
def bayesian_ridge(X_train, X_test, y_train, y_test, future_forecast, state):
        
    # convert data to be compatible with polynomial regression
    bayesian_poly = PolynomialFeatures(degree=3)
    bayesian_poly_X_train = bayesian_poly.fit_transform(X_train)
    bayesian_poly_X_test = bayesian_poly.fit_transform(X_test)
    bayesian_poly_future_forecast = bayesian_poly.fit_transform(future_forecast)
    
    # polynomial regression model
    # bayesian ridge polynomial regression
    tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
    alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
    alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
    lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
    lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
    normalize = [True, False]
    fit_intercept = [True,  False]
    lambda_init = [1e-2, 1e-1, 1, 1e1]

    bayesian_grid = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                    'normalize' : normalize, 'fit_intercept': fit_intercept, 'lambda_init' : lambda_init}

    bayesian = BayesianRidge()
    bayesian_search = RandomizedSearchCV(bayesian, bayesian_grid, scoring='neg_root_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=200, verbose=1)
    bayesian_search.fit(bayesian_poly_X_train, y_train)
    
    # get the best estimator 
    best_params = bayesian_search.best_params_
    bayesian_confirmed = BayesianRidge(**best_params)
    bayesian_confirmed.fit(bayesian_poly_X_train, y_train)
    
    test_bayesian_pred = bayesian_confirmed.predict(bayesian_poly_X_test)
    bayesian_pred = bayesian_confirmed.predict(bayesian_poly_future_forecast)
    
    plt.plot(y_test)
    plt.plot(test_bayesian_pred)
    plt.title('Testing Set Evaluation for {}'.format(state))
    plt.xlabel('Days since 3/1/2020')
    plt.ylabel('# of positive coronavirus cases')
    plt.legend(['Actual', 'Predicted'])
    plt.show()
    
    print('MAE:', mean_absolute_error(test_bayesian_pred, y_test))
    print('MSE:',mean_squared_error(test_bayesian_pred, y_test))
    print('Weight:', bayesian_confirmed.coef_)

    # plot the graph to see compare predictions and actual coronavirus cases
    plt.plot(positive)
    plt.plot(bayesian_pred)
    plt.title('Coronavirus Cases in {}'.format(state))
    plt.legend(['Actual cases', 'Predicted cases using bayesian ridge'])
    plt.xlabel('Days since 3/1/2020')
    plt.ylabel('# of positive coronavirus cases')
    plt.show()
    print('Completed:', state)


In [ ]:
# helper function for finding daily change 
def daily_change(y2, y1):
    return (y2-y1)

# We are taking moving averages for coronavirus cases (our label). It gives us a better picture than a single day value.

In [ ]:
# get moving average for positive case 

def moving_positive_cases(data, window_size):
    moving_positive = []
    for i in range(len(data)):
        if i + window_size < len(data):
            moving_positive.append(np.mean(data[i:i+window_size]))
        else:
            moving_positive.append(np.mean(data[i:len(data)]))
    return moving_positive

# We want to extrapolate testing information using a polynomial regression. This will provide us with values for future prediction. 

In [ ]:
def future_testing_extrapolation(X, y, future_forecast, state):
    poly = PolynomialFeatures(degree=3)
    poly_X = poly.fit_transform(X)
    poly_future_forecast = poly.fit_transform(future_forecast)
    
    poly_confirmed = LinearRegression(fit_intercept=True, normalize=True)
    poly_confirmed.fit(poly_X, y)
    
    poly_pred = poly_confirmed.predict(poly_future_forecast)
    
    plt.plot(y)
    plt.plot(poly_pred)
    plt.title('Coronavirus testing in {}'.format(state))
    plt.legend(['Actual testing', 'Predicted testing using polynomial regression'])
    plt.xlabel('Days since 3/1/2020')
    plt.ylabel('# of testing')
    plt.show()
    
    future_increases = [] 
    
    # calulate future rates of change 
    for i in range(days_ahead):
        c = len(X) - 1
        future_increases.append(daily_change(poly_pred[c+i+1], poly_pred[c+i]))
    return future_increases

# Reshape our arays for sklearn. 

In [ ]:
days_since_3_1 = np.array(days_since_3_1).reshape(-1, 1)
future_dates = np.array(future_dates).reshape(-1, 1)

# We are taking winodws averages for our mobility data. It will give us a better view of the situation.

In [ ]:
def window_average(window_size, data, method):
    avg_data = [] 
    date_length = len(data)
    
    for i in range(len(data)):
        remainder = i % window_size 
        if method == 'median':
            if i - remainder + window_size - 1 < date_length:
                avg_data.append(np.median(data[i-remainder:i-remainder+window_size-1]))
            else:
                delta = date_length % window_size 
                avg_data.append(np.median(data[date_length-delta-1:date_length-1]))
        elif method == 'mean':
             if i - remainder + window_size - 1 < date_length:
                avg_data.append(np.mean(data[i-remainder:i-remainder+window_size-1]))
             else:
                delta = date_length % window_size 
                avg_data.append(np.mean(data[date_length-delta-1:date_length-1]))
        else:
            warnings.warn('Methods can only be mean or median')
            
    return avg_data

# Determine if a day is a weekend or weekday. We want to separate weekend mobility from weekday mobility due to statistical differences.

In [ ]:
# returns true if it is a weekend, and false if it is a weekday 
def weekday_or_weekend(date):
    date_obj = datetime.datetime.strptime(str(date), '%Y%m%d')
    day_of_the_week =  date_obj.weekday()
    if (day_of_the_week+1) % 6 == 0 or (day_of_the_week+1) % 7 == 0:
        return True 
    else:
        return False 

In [ ]:
len(future_dates[-10:])

In [ ]:
# implementing this in the prediction in the future 
def mobility_scenario(mobility, mode):
    local_min = np.min(mobility)
    local_max = np.max(mobility)
    
    local_min_index = np.where(mobility==local_min)[0]
    local_max_index = np.where(mobility==local_max)[0]
    
    slope = (local_max - local_min) / (local_max_index - local_min_index)
    
#     plt.axhline(y=local_min, color='red')
#     plt.axhline(y=local_max, color='purple')
    
    # if mobility increases 
    
    if mode == 'decrease':
        # extrapolating mobility 
        m = mobility[-1] + slope 
        if m < local_min:
            future_mobility = local_min
        else:
            future_mobility = m
    
    # if mobility decreases 
    if mode == 'increase':
         # extrapolating mobility 
        m = mobility[-1] + np.abs(slope) 
        if m > local_max:
            future_mobility = local_max
        else:
            future_mobility = m
            
    return future_mobility

# Building the model. 

In [ ]:
states = ['FL', 'CA', 'GA', 'TX']

for state in states:
    positive = []
    pop_density = [] 
    testing = [] 
    
    # mobility data
    walking_weekday = [] 
    walking_weekend = [] 
    walking = []
    walking_weekday_window = 7
    walking_weekend_window = 7
    
    # adjust window size for mobility
    
    date_length = len(revised_unique_dates)
    
    # get cases in sequential order for each state
    for i in range(date_length):
        positive.append(new_data[new_data.date==unique_dates[i]][new_data.state==state].positive.sum())
        pop_density.append(new_data[new_data.state==state]['pop_dense'].max())
        testing.append(new_data[new_data.date==unique_dates[i]][new_data.state==state].totalTestResults.sum())
        
        # determines if it is a weekend or weekday 
        if weekday_or_weekend(unique_dates[i]): 
            walking_weekend.append(get_mobility_by_state('walking', state, revised_unique_dates[i]))
        else:
            walking_weekday.append(get_mobility_by_state('walking', state, revised_unique_dates[i]))
        
#         remainder = i % window_size 
#         if i - remainder + window_size < date_length:
#             walking.append(get_mobility_by_state('walking', state, revised_unique_dates[i-remainder], revised_unique_dates[i-remainder+window_size-1], 'median'))
#         else:
#             # if extrapolating use the mobility average from the last few days based on the window size
#             delta = date_length % window_size 
#             walking.append(get_mobility_by_state('walking', state, revised_unique_dates[date_length-delta-1], revised_unique_dates[date_length-1], 'median'))


    # remove any decreases in cum testing and positive cases
    for i in range(len(testing)):
        if i != 0:
            if testing[i] < testing[i-1]:
                testing[i] = testing[i-1]
            if positive[i] < positive[i-1]:
                positive[i] = positive[i-1]
    
    # remove 0 in mobility from both weekday and weekend data (there are few null values from Apple's mobility data)
    for i in range(len(walking_weekend)):       
        if walking_weekend[i] == 0 and i != 0:
            walking_weekend[i] = walking_weekend[i-1]
            
    for i in range(len(walking_weekday)):
        if walking_weekday[i] == 0 and i != 0:
            walking_weekday[i] = walking_weekday[i-1]
            
    
    # taking window average for mobility 
    walking_weekday_avg = window_average(7, walking_weekday, 'mean')
    walking_weekend_avg = window_average(7, walking_weekend, 'mean')

    
    # making sure the shape of the mobility arrays match 
    r_walking_weekday_avg = [] 
    r_walking_weekend_avg = [] 
    
    k = 0 
    j = 0 
    for i in range(date_length):
        if i % walking_weekday_window == 0 and i != 0:
            if k + walking_weekday_window < len(walking_weekday_avg):
                k += walking_weekday_window
            else:
                k = len(walking_weekday_avg) - 1 
                
            if j + walking_weekend_window < len(walking_weekend_avg):
                j += walking_weekend_window
            else:
                j = len(walking_weekend_avg) - 1
        
        r_walking_weekday_avg.append(walking_weekday_avg[k])
        r_walking_weekend_avg.append(walking_weekend_avg[j])
        

    # take moving average for positive cases
    positive = moving_positive_cases(positive, 3)

    # future testing extrapolations from poylnomial prediction 
    future_testing = future_testing_extrapolation(days_since_3_1, testing, future_dates, state)
    for i in future_testing:
        testing.append(testing[-1] + i)
    
    testing = np.array(testing).reshape(-1, 1)
    positive = np.array(positive).reshape(-1, 1)
    r_walking_weekday_avg = np.array(r_walking_weekday_avg).reshape(-1, 1)
    r_walking_weekend_avg = np.array(r_walking_weekend_avg).reshape(-1, 1)
    
    min_max_scaler = MinMaxScaler()
    testing = min_max_scaler.fit_transform(testing)
    r_walking_weekday_avg = min_max_scaler.fit_transform(r_walking_weekday_avg)
    r_walking_weekend_avg = min_max_scaler.fit_transform(r_walking_weekend_avg)
    
    # combining the two features
    X = [] 
    future_forecast = []
    
    for i in range(len(days_since_3_1)):
        X.append([days_since_3_1[i][0], pop_density[0], testing[i][0], r_walking_weekday_avg[i][0], r_walking_weekend_avg[i][0]])
    
    X = np.array(X, object).reshape(-1, 5)
    
    for i in range(len(future_dates)):
        if i < date_length:
            future_forecast.append([future_dates[i][0], pop_density[0], testing[i][0], r_walking_weekday_avg[i][0], r_walking_weekend_avg[i][0]])
        else:
            future_forecast.append([future_dates[i][0], pop_density[0], testing[i][0], r_walking_weekday_avg[-1][0], r_walking_weekend_avg[-1][0]])
            
    future_forecast = np.array(future_forecast, object).reshape(-1, 5)
    
    # splitting into training and testing sets 
    X_train, X_test, y_train, y_test = train_test_split(X, positive, shuffle=False, test_size=0.05)
    bayesian_ridge(X_train, X_test, y_train, y_test, future_forecast, state)